## Advanced topics in Int. Trade. 
The goal of this script is to build the dataset for the project of the course Advanced Topics in International Trade

At first, we want to get data for the trade between countries. 

In [56]:
# import libraries
import os
import numpy as np
import pandas as pd

In [57]:
# arguments
exports_filename_2022_2017 = "exports_TradeData_6_11_2023_17_22_19.csv"
imports_filename_2022_2017 = "imports_TradeData_6_11_2023_17_25_3.csv"

exports_filename_2016_2011 = "exports_TradeData_6_11_2023_17_40_30.csv"
imports_filename_2016_2011 = "imports_TradeData_6_11_2023_17_31_55.csv"

trade_filename = "data.dta"
gravity_filename = r"gravity_1995_2014.dta"

country_code_filename = "iso-country-codes.csv"

In [58]:
folder_path = os.path.dirname(os.getcwd())
path_to_data = os.path.join(folder_path,'data')
path_to_data

'/Users/clement/Desktop/ULB/Doctorat/ECARES M2/Advanced topics in Economics/Project/Gravity_trade_2/data'

In [59]:
os.getcwd()

'/Users/clement/Desktop/ULB/Doctorat/ECARES M2/Advanced topics in Economics/Project/Gravity_trade_2/src'

#### Load data

In [60]:
country_code = pd.read_csv(os.path.join(path_to_data, country_code_filename))
country_code.rename(columns = {'English short name lower case' : 'Country'}, inplace = True)
country_code = country_code[['Country', 'Alpha-3 code']]

In [61]:
mapping_years = {'2012 [YR2012]': '2012',
'2013 [YR2013]': '2013',
'2014 [YR2014]': '2014',
'2015 [YR2015]': '2015',
'2016 [YR2016]': '2016',
'2017 [YR2017]': '2017',
'2018 [YR2018]': '2018',
'2019 [YR2019]': '2019',
'2020 [YR2020]': '2020',
'2021 [YR2021]': '2021',
'2022 [YR2022]': '2022'
}

##### 1.1 Trade data 

In [62]:
gravity = pd.read_stata(os.path.join(path_to_data, gravity_filename))
gravity.t = gravity.t.astype(int)

In [63]:
# I need to join the data from 2015 to 2022
distance = gravity[['j','i','dist']].drop_duplicates()
distance.rename(columns={'i':'ReporterISO','j':'PartnerISO'}, inplace=True)

In [64]:
exports_2022_2017_trade = pd.read_csv(os.path.join(path_to_data,exports_filename_2022_2017), sep=',', encoding='latin-1')
exports_2016_2015_trade = pd.read_csv(os.path.join(path_to_data,exports_filename_2016_2011), sep=',',encoding='latin-1')
exports_2016_2015_trade = exports_2016_2015_trade[(exports_2016_2015_trade['RefYear'] == 2015) | (exports_2016_2015_trade['RefYear'] == 2016)]
exports_2022_2017_trade = exports_2022_2017_trade.loc[:, ['RefYear','ReporterISO', 'PartnerISO', 'PrimaryValue']]
exports_2016_2015_trade = exports_2016_2015_trade.loc[:, ['RefYear','ReporterISO', 'PartnerISO', 'PrimaryValue']]
exports_2014_1995_trade = gravity.loc[:, ['t','i', 'j', 'v']]
exports_2014_1995_trade.rename(columns={'t': 'RefYear','i':'ReporterISO','j':'PartnerISO','v':'PrimaryValue'}, inplace=True)
exports_2022_1995 = pd.concat([exports_2014_1995_trade, exports_2016_2015_trade, exports_2022_2017_trade], ignore_index=True)

In [65]:
# For each pair of country, join the distance
exports_2022_1995 = pd.merge(exports_2022_1995, distance, on=['ReporterISO', 'PartnerISO'], how='left')

##### 1.3 Add GDP weights

In [66]:
# I need to join the data from 2015 to 2022
gdp_1995_2014 = gravity[['t','j','i','gdp_i','gdp_j']].drop_duplicates()
gdp_1995_2014.rename(columns={'t':'year','i':'ReporterISO','j':'PartnerISO'}, inplace=True)

In [67]:
gdp_1995_2014

,year,PartnerISO,ReporterISO,gdp_i,gdp_j
0,2008,AIA,ABW,2.791961e+09,NaN
1,1996,ALB,ABW,1.379888e+09,3.314898e+09
2,1997,AND,ABW,1.531844e+09,1.180597e+09
3,2005,ANT,ABW,2.331006e+09,NaN
4,2006,ANT,ABW,2.421475e+09,NaN
...,...,...,...,...,...
484766,2010,ZMB,ZWE,1.005205e+10,2.026556e+10
484767,2011,ZMB,ZWE,1.207173e+10,2.346010e+10
484768,2012,ZMB,ZWE,1.405838e+10,2.550337e+10
484769,2013,ZMB,ZWE,1.522353e+10,2.804546e+10


In [68]:
gdp_1995_2014.groupby(['year','PartnerISO']).count()

ReporterISO  gdp_i  gdp_j
year PartnerISO                           
1995 ABW                  32     32     32
     AFG                  51     51      0
     AGO                  61     61     61
     AIA                  27     27      0
     ALB                  51     51     51
...                      ...    ...    ...
2014 YEM                 127    118    127
     ZAF                 203    176    203
     ZAR                  88     87      0
     ZMB                 145    128    145
     ZWE                 121    110    121

[4288 rows x 3 columns]

In [69]:
gdp1_filename = r"b96863f7-3fb5-4cdf-aefb-b60d5c0e9ef5_Data.csv"
gdp2_filename = r"b96863f7-3fb5-4cdf-aefb-b60d5c0e9ef5_Series - Metadata.csv"


In [70]:
# Load gdp data
def prepare_gdp_file(path_to_data, gdp_filename): 
    """Refactofing file.

    Args:
        path_to_data (_type_): _description_
        gdp_filename (_type_): _description_
    """
    gdp1 = pd.read_csv(os.path.join(path_to_data, gdp_filename),
    dtype={'GDP (current US$)':int}
    )
    gdp1 = gdp1[gdp1["Series Name"] == "GDP (current US$)"]
    gdp1.rename(columns=mapping_years, inplace=True)
    gdp1 = gdp1.drop(columns=['Country Name','Series Name', 'Series Code'])
    gdp1 = gdp1.set_index('Country Code')
    gdp1 = gdp1.stack().reset_index()
    gdp1.rename(columns = {'level_1':'RefYear', 0:'gdp'}, inplace=True)
    gdp1['RefYear'] = gdp1['RefYear'].astype(int)
    gdp1 = gdp1[gdp1['gdp'] != '..']
    gdp1['gdp'] = gdp1['gdp'].astype(float) 
    
    return gdp1

In [71]:
gdp1 = prepare_gdp_file(path_to_data, gdp1_filename)
gdp2 = prepare_gdp_file(path_to_data, gdp2_filename)
gdp = pd.concat([gdp1, gdp2], ignore_index=True)

In [72]:
gdp_1995_2014.rename({'ReporterISO':'Country Code', 'gdp_i':'gdp','year':'RefYear'}, axis = 1, inplace = True)
gdp_1995_2014 = gdp_1995_2014[['Country Code', 'RefYear','gdp']]
gdp_1995_2014


,Country Code,RefYear,gdp
0,ABW,2008,2.791961e+09
1,ABW,1996,1.379888e+09
2,ABW,1997,1.531844e+09
3,ABW,2005,2.331006e+09
4,ABW,2006,2.421475e+09
...,...,...,...
484766,ZWE,2010,1.005205e+10
484767,ZWE,2011,1.207173e+10
484768,ZWE,2012,1.405838e+10
484769,ZWE,2013,1.522353e+10


In [73]:
gdp = pd.concat([gdp, gdp_1995_2014], ignore_index=True)

In [74]:
gdp = gdp.drop_duplicates()

In [75]:
# For each exporter country, join the yearly GDP
exports_2022_1995 = pd.merge(exports_2022_1995, gdp, left_on=['ReporterISO', 'RefYear'], right_on=['Country Code', 'RefYear'], how='left')
exports_2022_1995.rename(columns={'gdp':'gdp_i'}, inplace=True)

In [76]:
# For each exporter country, join the yearly GDP
exports_2022_1995 = pd.merge(exports_2022_1995, gdp, left_on=['PartnerISO', 'RefYear'], right_on=['Country Code', 'RefYear'], how='left')
exports_2022_1995.rename(columns={'gdp':'gdp_j'}, inplace=True)

In [77]:
exports_2022_1995.drop('Country Code_x', axis=1, inplace=True)
exports_2022_1995.drop('Country Code_y', axis=1, inplace=True)

##### 2.2  Add BRI_membership


In [78]:
wto_filename = r"mem-obs-list.xlsx"
bri_membership = pd.read_excel(os.path.join(path_to_data, wto_filename), sheet_name='BRI')
# add country code !
bri_membership = pd.merge(bri_membership, country_code, left_on=['Pays'], right_on=['Country'], how='left')
bri_membership.drop(['Pays', 'Region','Income','Country','IsinWTOFILe'], axis = 1, inplace = True)

bri_membership['Date'] = bri_membership['Date'].astype(int)

In [79]:
bri_membership

,Date,Alpha-3 code
0,2013,AFG
1,2017,ALB
2,2018,DZA
3,2018,AGO
4,2018,ATG
...,...,...
134,2018,VEN
135,2017,VNM
136,2017,YEM
137,2018,ZMB


In [80]:
exports_2022_1995 = pd.merge(exports_2022_1995, bri_membership, left_on=['ReporterISO'], right_on=['Alpha-3 code'], how='left')
# Compare the dates to determine if each row is after the membership date
exports_2022_1995['BRI_i'] = exports_2022_1995['RefYear'] >= exports_2022_1995['Date']
exports_2022_1995.drop(['Date','Alpha-3 code'], axis = 1, inplace = True)

In [81]:
exports_2022_1995 = pd.merge(exports_2022_1995, bri_membership, left_on=['PartnerISO'], right_on=['Alpha-3 code'], how='left')
# Compare the dates to determine if each row is after the membership date
exports_2022_1995['BRI_j'] = exports_2022_1995['RefYear'] >= exports_2022_1995['Date']
exports_2022_1995.drop(['Date','Alpha-3 code'], axis = 1, inplace = True)

In [82]:
exports_2022_1995

,RefYear,ReporterISO,PartnerISO,PrimaryValue,dist,gdp_i,gdp_j,BRI_i,BRI_j
0,2008,ABW,AIA,5.084000e+03,983.2682,2.791961e+09,NaN,False,False
1,1996,ABW,ALB,1.533525e+04,9091.7420,1.379888e+09,3.314898e+09,False,False
2,1997,ABW,AND,1.174000e+03,7572.7880,1.531844e+09,1.180597e+09,False,False
3,2005,ABW,ANT,2.638984e+07,136.3848,2.331006e+09,NaN,False,False
4,2006,ABW,ANT,2.400926e+07,136.3848,2.421475e+09,NaN,False,False
...,...,...,...,...,...,...,...,...,...
814529,2022,UZB,TZA,6.375000e+04,6163.4070,NaN,NaN,True,True
814530,2022,UZB,USA,1.655208e+07,10179.6600,NaN,NaN,True,False
814531,2022,UZB,VEN,4.476500e+04,12722.7600,NaN,NaN,True,True
814532,2022,UZB,YEM,4.488000e+04,3750.1890,NaN,NaN,True,True


In [83]:
exports_2022_1995['BRI_i'] = exports_2022_1995['BRI_i'].replace({True: 1, False: 0})
exports_2022_1995['BRI_j'] = exports_2022_1995['BRI_j'].replace({True: 1, False: 0})

In [84]:
exports_2022_1995

,RefYear,ReporterISO,PartnerISO,PrimaryValue,dist,gdp_i,gdp_j,BRI_i,BRI_j
0,2008,ABW,AIA,5.084000e+03,983.2682,2.791961e+09,NaN,0,0
1,1996,ABW,ALB,1.533525e+04,9091.7420,1.379888e+09,3.314898e+09,0,0
2,1997,ABW,AND,1.174000e+03,7572.7880,1.531844e+09,1.180597e+09,0,0
3,2005,ABW,ANT,2.638984e+07,136.3848,2.331006e+09,NaN,0,0
4,2006,ABW,ANT,2.400926e+07,136.3848,2.421475e+09,NaN,0,0
...,...,...,...,...,...,...,...,...,...
814529,2022,UZB,TZA,6.375000e+04,6163.4070,NaN,NaN,1,1
814530,2022,UZB,USA,1.655208e+07,10179.6600,NaN,NaN,1,0
814531,2022,UZB,VEN,4.476500e+04,12722.7600,NaN,NaN,1,1
814532,2022,UZB,YEM,4.488000e+04,3750.1890,NaN,NaN,1,1


##### Add WTO and OECD membership

In [85]:
wto_filename = r"mem-obs-list.xlsx"
wto_membership = pd.read_excel(os.path.join(path_to_data, wto_filename))
# add country code !
wto_membership = pd.merge(wto_membership, country_code, left_on=['Members'], right_on=['Country'], how='left')
wto_membership['wto_member'] = np.ones(len(wto_membership)).astype('int')
wto_membership.drop(['Members','Country'],axis = 1, inplace = True)
wto_membership['wto_membership_year'] = pd.to_datetime(wto_membership['Membership Date'], format='%d %B %Y').dt.year
wto_membership['wto_membership_year'].fillna(0, inplace=True)
wto_membership['wto_membership_year'] = wto_membership['wto_membership_year'].astype(int)
wto_membership.dropna(subset=['Membership Date'], inplace=True)
wto_membership.drop('Membership Date',axis = 1,inplace = True)
wto_membership.head()

,Alpha-3 code,wto_member,wto_membership_year
0,AFG,1,2016
1,ALB,1,2000
2,AGO,1,1996
3,ATG,1,1995
4,ARG,1,1995


In [86]:
exports_2022_1995 = pd.merge(exports_2022_1995, wto_membership, left_on=['ReporterISO'], right_on=['Alpha-3 code'], how='left')
# Compare the dates to determine if each row is after the membership date
exports_2022_1995['WTO_i'] = exports_2022_1995['RefYear'] >= exports_2022_1995['wto_membership_year']

In [87]:
exports_2022_1995.drop('wto_membership_year',axis = 1,inplace = True)

In [88]:
# Same with importer
exports_2022_1995 = pd.merge(exports_2022_1995, wto_membership, left_on=['PartnerISO'], right_on=['Alpha-3 code'], how='left')
# Compare the dates to determine if each row is after the membership date
exports_2022_1995['WTO_j'] = exports_2022_1995['RefYear'] >= exports_2022_1995['wto_membership_year']
exports_2022_1995.drop(['Alpha-3 code_x','wto_member_x','Alpha-3 code_y','wto_member_y','wto_membership_year'], axis = 1, inplace = True)

In [89]:
exports_2022_1995.columns

Index(['RefYear', 'ReporterISO', 'PartnerISO', 'PrimaryValue', 'dist', 'gdp_i',
       'gdp_j', 'BRI_i', 'BRI_j', 'WTO_i', 'WTO_j'],
      dtype='object')

In [90]:
exports_2022_1995['WTO_i'] = exports_2022_1995['WTO_i'].replace({True: 1, False: 0})
exports_2022_1995['WTO_j'] = exports_2022_1995['WTO_j'].replace({True: 1, False: 0})

##### Add OECD membership

In [91]:
country_code['COUNTRY'] = country_code['Country'].str.upper()

In [92]:
wto_filename = r"mem-obs-list.xlsx"
oecd_membership = pd.read_excel(os.path.join(path_to_data, wto_filename), sheet_name='oecd')
# add country code !
oecd_membership = pd.merge(oecd_membership, country_code, left_on=['country'], right_on=['COUNTRY'], how='left')
oecd_membership['oecd_member'] = np.ones(len(oecd_membership)).astype('int')
oecd_membership.drop(['country'],axis = 1, inplace = True)
oecd_membership['oecd_membership_year'] = pd.to_datetime(oecd_membership['date'], format='%d %B %Y').dt.year
oecd_membership['oecd_membership_year'].fillna(0, inplace=True)
oecd_membership['oecd_membership_year'] = oecd_membership['oecd_membership_year'].astype(int)
oecd_membership.dropna(subset=['date'], inplace=True)
oecd_membership.drop('date',axis = 1,inplace = True)
oecd_membership.head()

,Country,Alpha-3 code,COUNTRY,oecd_member,oecd_membership_year
0,Australia,AUS,AUSTRALIA,1,1971
1,Austria,AUT,AUSTRIA,1,1961
2,Belgium,BEL,BELGIUM,1,1961
3,Canada,CAN,CANADA,1,1961
4,Chile,CHL,CHILE,1,2010


In [93]:
exports_2022_1995 = pd.merge(exports_2022_1995, oecd_membership, left_on=['ReporterISO'], right_on=['Alpha-3 code'], how='left')
# Compare the dates to determine if each row is after the membership date
exports_2022_1995['OECD_i'] = exports_2022_1995['RefYear'] >= exports_2022_1995['oecd_membership_year']
exports_2022_1995.drop(['Country', 'Alpha-3 code', 'COUNTRY', 'oecd_member', 'oecd_membership_year'],axis = 1, inplace = True)

In [94]:
exports_2022_1995 = pd.merge(exports_2022_1995, oecd_membership, left_on=['PartnerISO'], right_on=['Alpha-3 code'], how='left')
# Compare the dates to determine if each row is after the membership date
exports_2022_1995['OECD_j'] = exports_2022_1995['RefYear'] >= exports_2022_1995['oecd_membership_year']

In [95]:
exports_2022_1995.drop(['Country', 'Alpha-3 code', 'COUNTRY', 'oecd_member', 'oecd_membership_year'],axis = 1, inplace = True)

##### Add BRICS

In [96]:
wto_filename = r"mem-obs-list.xlsx"
brics_membership = pd.read_excel(os.path.join(path_to_data, wto_filename), sheet_name='brics')
# add country code !
brics_membership = pd.merge(brics_membership, country_code, on=['Country'], how='left')
brics_membership.drop(['Country'],axis = 1, inplace = True)
brics_membership['brics_membership_year'] = pd.to_datetime(brics_membership['Date'], format='%Y').dt.year
brics_membership['brics_membership_year'].fillna(0, inplace=True)
brics_membership['brics_membership_year'] = brics_membership['brics_membership_year'].astype(int)
brics_membership.dropna(subset=['Date'], inplace=True)
brics_membership.drop('Date',axis = 1,inplace = True)
brics_membership.head()

,Alpha-3 code,COUNTRY,brics_membership_year
0,RUS,RUSSIA,2009
1,CHN,CHINA,2009
2,IND,INDIA,2009
3,MEX,MEXICO,2011
4,BRA,BRAZIL,2009


In [97]:
exports_2022_1995 = pd.merge(exports_2022_1995, brics_membership, left_on=['ReporterISO'], right_on=['Alpha-3 code'], how='left')
# Compare the dates to determine if each row is after the membership date
exports_2022_1995['BRICS_i'] = exports_2022_1995['RefYear'] >= exports_2022_1995['brics_membership_year']

In [98]:
exports_2022_1995.columns

Index(['RefYear', 'ReporterISO', 'PartnerISO', 'PrimaryValue', 'dist', 'gdp_i',
       'gdp_j', 'BRI_i', 'BRI_j', 'WTO_i', 'WTO_j', 'OECD_i', 'OECD_j',
       'Alpha-3 code', 'COUNTRY', 'brics_membership_year', 'BRICS_i'],
      dtype='object')

In [99]:
exports_2022_1995.drop([ 'Alpha-3 code', 'COUNTRY','brics_membership_year'], axis = 1, inplace=True)

In [100]:
exports_2022_1995 = pd.merge(exports_2022_1995, brics_membership, left_on=['PartnerISO'], right_on=['Alpha-3 code'], how='left')
# Compare the dates to determine if each row is after the membership date
exports_2022_1995['BRICS_j'] = exports_2022_1995['RefYear'] >= exports_2022_1995['brics_membership_year']

In [101]:
exports_2022_1995.drop(['Alpha-3 code', 'COUNTRY', 'brics_membership_year'], axis = 1, inplace=True)

In [102]:
exports_2022_1995['OECD_i'] = exports_2022_1995['OECD_i'].replace({True: 1, False: 0})
exports_2022_1995['OECD_j'] = exports_2022_1995['OECD_j'].replace({True: 1, False: 0})
exports_2022_1995['BRICS_i'] = exports_2022_1995['BRICS_i'].replace({True: 1, False: 0})
exports_2022_1995['BRICS_j'] = exports_2022_1995['BRICS_j'].replace({True: 1, False: 0})

##### Compute other dummmies

In [103]:
exports_2022_1995['BRI_OECD'] = (exports_2022_1995['BRI_i'] == 1) & (exports_2022_1995['BRI_j'] == 1) & (exports_2022_1995['OECD_i'] == 1) & (exports_2022_1995['OECD_j'] == 1)
exports_2022_1995['BRI_OECD'] = exports_2022_1995['BRI_OECD'].astype(int)
# SAME WITH WTO
exports_2022_1995['BRI_OECD'] = (exports_2022_1995['BRI_i'] == 1) & (exports_2022_1995['BRI_j'] == 1) & (exports_2022_1995['BRICS_i'] == 1) & (exports_2022_1995['BRICS_j'] == 1)
exports_2022_1995['BRI_OECD'] = exports_2022_1995['BRI_OECD'].astype(int)

from pandas_profiling import ProfileReport
# Generate the ProfileReport
profile = ProfileReport(exports_2022_1995, title="Pandas Profiling Report", explorative=True)

# Save the report as an HTML file
path_to_save = os.path.join('dataset' + ".html")
profile.to_file(path_to_save)


In [104]:
exports_2022_1995.rename({'PartnerISO':'i','ReporterISO':'j','PrimaryValue':'v'}, axis=1, inplace=True)

In [105]:
exports_2022_1995

,RefYear,j,i,v,dist,gdp_i,gdp_j,BRI_i,BRI_j,WTO_i,WTO_j,OECD_i,OECD_j,BRICS_i,BRICS_j,BRI_OECD
0,2008,ABW,AIA,5.084000e+03,983.2682,2.791961e+09,NaN,0,0,0,0,0,0,0,0,0
1,1996,ABW,ALB,1.533525e+04,9091.7420,1.379888e+09,3.314898e+09,0,0,0,0,0,0,0,0,0
2,1997,ABW,AND,1.174000e+03,7572.7880,1.531844e+09,1.180597e+09,0,0,0,0,0,0,0,0,0
3,2005,ABW,ANT,2.638984e+07,136.3848,2.331006e+09,NaN,0,0,0,0,0,0,0,0,0
4,2006,ABW,ANT,2.400926e+07,136.3848,2.421475e+09,NaN,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
814529,2022,UZB,TZA,6.375000e+04,6163.4070,NaN,NaN,1,1,0,0,0,0,0,0,0
814530,2022,UZB,USA,1.655208e+07,10179.6600,NaN,NaN,1,0,0,1,0,0,0,0,0
814531,2022,UZB,VEN,4.476500e+04,12722.7600,NaN,NaN,1,1,0,1,0,0,0,0,0
814532,2022,UZB,YEM,4.488000e+04,3750.1890,NaN,NaN,1,1,0,1,0,0,0,0,0


In [106]:
gravity.columns

Index(['t', 'i', 'j', 'v', 'gdp_i', 'gdp_j', 'dist', 'gdppc_i', 'wto_i',
       'gdppc_j', 'wto_j', 'rta', 'cu', 'fta', 'eia', 'ps', 'cuandeia',
       'ftaandeia', 'contig', 'comlang_off', 'comlang_ethno', 'colony',
       'comcol'],
      dtype='object')

In [107]:
gravity[['i','j','rta', 'cu', 'fta', 'eia', 'ps', 'cuandeia',
       'ftaandeia', 'contig', 'comlang_off', 'comlang_ethno', 'colony',
       'comcol']]

,i,j,rta,cu,fta,eia,ps,cuandeia,ftaandeia,contig,comlang_off,comlang_ethno,colony,comcol
0,ABW,AIA,1,1,0,1,0,1,0,0.0,0.0,1.0,0.0,0.0
1,ABW,ALB,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
2,ABW,AND,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0
3,ABW,ANT,1,1,0,1,0,1,0,0.0,1.0,1.0,0.0,1.0
4,ABW,ANT,1,1,0,1,0,1,0,0.0,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484766,ZWE,ZMB,1,1,1,0,0,0,0,1.0,1.0,1.0,0.0,1.0
484767,ZWE,ZMB,1,1,1,0,0,0,0,1.0,1.0,1.0,0.0,1.0
484768,ZWE,ZMB,1,1,1,0,0,0,0,1.0,1.0,1.0,0.0,1.0
484769,ZWE,ZMB,1,1,1,0,0,0,0,1.0,1.0,1.0,0.0,1.0


In [108]:
exports_2022_1995 = pd.merge(exports_2022_1995, gravity[['i','j','rta', 'cu', 'fta', 'eia', 'ps', 'cuandeia',
       'ftaandeia', 'contig', 'comlang_off', 'comlang_ethno', 'colony',
       'comcol']], left_on=['i','j'], right_on=['i','j'], how='left')

In [109]:
exports_2022_1995

,RefYear,j,i,v,dist,gdp_i,gdp_j,BRI_i,BRI_j,WTO_i,...,fta,eia,ps,cuandeia,ftaandeia,contig,comlang_off,comlang_ethno,colony,comcol
0,2008,ABW,AIA,5.084000e+03,983.2682,2.791961e+09,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1996,ABW,ALB,1.533525e+04,9091.7420,1.379888e+09,3.314898e+09,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1997,ABW,AND,1.174000e+03,7572.7880,1.531844e+09,1.180597e+09,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,2005,ABW,ANT,2.638984e+07,136.3848,2.331006e+09,NaN,0,0,0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0
4,2005,ABW,ANT,2.638984e+07,136.3848,2.331006e+09,NaN,0,0,0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12680515,2022,UZB,USA,1.655208e+07,10179.6600,NaN,NaN,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12680516,2022,UZB,USA,1.655208e+07,10179.6600,NaN,NaN,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12680517,2022,UZB,VEN,4.476500e+04,12722.7600,NaN,NaN,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12680518,2022,UZB,YEM,4.488000e+04,3750.1890,NaN,NaN,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
